## Задача 1
В файле gsm_list.txt содержатся идентификаторы GEO NCBI (идентификаторы вида GSM#####). Вам необходимо подгрузить html-страницу с информацией по каждому эксперименту. Например, https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM5176077. Обратите внимание на то, как устроена URL-ссылка. Ознакомтесь со страницей и имеющейся на ней информацией. Напишите программу, которая ищет соответствие серии экспериментов и эксперимента. Результат запищите ввиде таблицы, состоящей из 2 колонок.

\* В базе данных GEO NCBI некоторые из серий экспермиентов являются суперсериями, т. е. включают в себя другие серии экспериментов. Информацию об этом можно найти на соответствующих им страницам. Модифицируйте вашу программу так, чтобы идентификаторы суперсерий игнорировались

In [33]:
# Место для решения

import os
path_to_code = os.getcwd()
path_to_inp_file = os.path.join(path_to_code, os.pardir, 'lections', 'files_and_images', 'gsm_list.txt')

with open(path_to_inp_file, 'r') as inp_file:
    list_of_id = []
    for line in inp_file:
        line = line.rstrip()
        list_of_id.append(line)
list_of_id

import json
import requests

#response = requests.get('https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM2912912')
response = requests.get(f'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={list_of_id[0]}')
#print(response.text)

import bs4
#list_of_html = []
response_to_bs = bs4.BeautifulSoup(response.text, "html.parser")
print(response_to_bs)
list_of_p = response_to_bs('p')
print(list_of_p)
list_of_series = response_to_bs('Series')
print(list_of_series)

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<style type="text/css">
      a { text-decoration: none; }
	.bordergc {background-color: #6699CC;}
	.bordergd {background-color: #B6C7E5;}
	.borderge {background-color: #EEF3FB;}
	.bordergf {background-color: #FFFFFF;}
	.bordergg {background-color: #CCCCCC;}
      .small8b { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#6633cc;
              }
      .small8db { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#4411aa;
              }

    </style>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="NCBI GEO Gene Expression Omnibus microarray oligonucleotide array SAGE" name="keywords"/>
<meta content="NCBI's Gene Expression Omnibus (GEO) is a public archive and resource for gene expression data." name="description"/>
<meta content="geo" name="ncbi_app"/>
<meta content="full" name="ncb

## Задача 2
В файле encode_id_list.txt содержатся идентификаторы базы данных ENCODE (идентификаторы с префиксом ENCSR). Вам необходимо подгрузить json с информацией по каждому эксперименту. Например, https://www.encodeproject.org/experiments/ENCSR747VUU/?format=json. Обратите внимание на то, как устроена URL-ссылка. Ознакомтесь с JSON'ом и имеющейся в нём информацией. Напишите программу, которая собирает информацию об типе эксперимента (assay_title), таргете (target -> label) и дате сабмита данных (date_submitted) для каждого идентификатора. Результат представьте в виде таблицы, состоящей из 4 колонок.

In [2]:
# Место для решения